In [16]:
import pandas as pd
import numpy as np


In [54]:
sat = pd.read_excel('data/UCS-Satellite-Database.xlsx')
sat.head()
print(sat.shape)
sat.dropna(axis=1, how='all', inplace=True)

(6718, 67)


In [49]:
useable_data = sat.columns[0:24]
sat_cleaned = sat[useable_data]
sat_cleaned.head()

,Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,Longitude of GEO (degrees),...,Period (minutes),Launch Mass (kg.),Dry Mass (kg.),Power (watts),Date of Launch,Expected Lifetime (yrs.),Contractor,Country of Contractor,Launch Site,Launch Vehicle
0,1HOPSAT-TD,NR,USA,Hera Systems,Commercial,Earth Observation,Infrared Imaging,LEO,Non-Polar Inclined,0.0,...,96.08,22.0,NaN,NaN,2019-12-11 00:00:00,0.5,Hera Systems,USA,Satish Dhawan Space Centre,PSLV
1,Aalto-1,Finland,Finland,Aalto University,Civil,Technology Development,NaN,LEO,Sun-Synchronous,0.0,...,94.70,4.5,NaN,4.5,2017-06-23 00:00:00,2.0,Aalto University,Finland,Satish Dhawan Space Centre,PSLV
2,AAt-4,Denmark,Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,0.0,...,95.90,1.0,NaN,NaN,2016-04-25 00:00:00,NaN,University of Aalborg,Denmark,Guiana Space Center,Soyuz-2.1a
3,ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.0,...,1436.03,6330.0,NaN,16000,2014-02-06 00:00:00,15.0,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA
4,ABS-2A,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,-75.0,...,1436.10,1800.0,NaN,NaN,2016-06-15 00:00:00,15.0,Boeing Satellite Systems,USA,Cape Canaveral,Falcon 9


In [55]:
sat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6718 entries, 0 to 6717
Data columns (total 67 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Current Official Name of Satellite  6718 non-null   object 
 1   Country/Org of UN Registry          6718 non-null   object 
 2   Country of Operator/Owner           6718 non-null   object 
 3   Operator/Owner                      6718 non-null   object 
 4   Users                               6718 non-null   object 
 5   Purpose                             6718 non-null   object 
 6   Detailed Purpose                    1167 non-null   object 
 7   Class of Orbit                      6718 non-null   object 
 8   Type of Orbit                       6077 non-null   object 
 9   Longitude of GEO (degrees)          6716 non-null   float64
 10  Perigee (km)                        6718 non-null   int64  
 11  Apogee (km)                         6718 no

In [51]:
sat_cleaned.isna().sum()

USA                           4511
China                          586
United Kingdom                 561
Russia                         177
Japan                           88
                              ... 
United Kingdom/Netherlands       1
Laos                             1
China                            1
USA/Canada                       1
China/Italy                      1
Name: Country of Operator/Owner, Length: 104, dtype: int64